In [275]:
# Modules & Libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy import stats

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [276]:
# Data
df = pd.read_csv("/content/drive/MyDrive/0_Hackathon_SBSSU/liver cd/resampling_approach/ilpd_md2.csv")
df.head()

,a_g_ratio,gender,age,t_bilirubin,d_bilirubin,alkphos,sgpt,sgot,t_protein,albumin,result
0,0.90,0,65.0,0.7,0.10,187.0,16.0,18.0,6.8,3.3,1
1,0.74,1,62.0,5.3,2.95,481.0,64.0,100.0,7.5,3.2,1
2,0.89,1,62.0,5.3,2.95,481.0,60.0,68.0,7.0,3.3,1
3,1.00,1,58.0,1.0,0.40,182.0,14.0,20.0,6.8,3.4,1
4,0.40,1,72.0,3.9,2.00,195.0,27.0,59.0,7.3,2.4,1


In [277]:
# Problem of Uneven Distribution
total_data = df.shape[0]
print(total_data)
Liver_patients_data = df[df['result']==1].shape[0]
Non_Liver_patients_data = df[df['result']==2].shape[0]

Liver_patients_data = Liver_patients_data / total_data * 100
Non_Liver_patients_data = Non_Liver_patients_data / total_data * 100

print("%.0f percent i.e. 406 Liver patients data\n%.0f precent i.e. 164 Non Liver Patients data"%(Liver_patients_data, Non_Liver_patients_data))

570
71 percent i.e. 406 Liver patients data
29 precent i.e. 164 Non Liver Patients data


As a result of which the model will be biased towards the highly distributed data i.e liver patients data, this needs to be resolved

we will make 10 dataset, each having RANDOMLY SELECTED 164 Liver patients dataset and 164 Non liver patients dataset

using DATAFRAME.SAMPLE() and PD.CONCAT([df1,df2])

In [280]:
# Original Data (both classes)
lpd =  df[df['result']==1]
nlpd = df[df['result']==2]

# Making Unseen data (20%) = 57 & 57
unseen_lpd  = lpd.iloc[:28,:]
unseen_nlpd = nlpd.iloc[:28,:]
unseen = pd.concat([unseen_lpd,unseen_nlpd])
unseen = unseen.sample(56)

unseen_x = unseen.drop(['result'],axis=1)
unseen_y = unseen['result']

# Making Data for Model Building
lpd  = lpd.iloc[28:,:]
nlpd = nlpd.iloc[28:,:]

In [281]:
# CReating 10 dataframes from model building data **
## 

dfs = []
for i in range(50):
  df1 = pd.concat([lpd.sample(378),nlpd.sample(100)])
  df1  = df1.sample(478)
  dfs.append(df1)

10 Dataframes are ready to use

In [282]:

def train_model_dt(df):
  x_train = df.drop(['result'],axis=1)
  y_train = df['result']

  dt = RandomForestClassifier(max_depth=6)
  dt.fit(x_train,y_train)
  return dt

def train_final_model_dt(dfs,x_test,y_test):
  models = []
  for df in dfs:
    model = train_model_dt(df)
    models.append(model)

  # Prediction
  y_pred_df = pd.DataFrame()
  for model in models:
    y_pred = model.predict(x_test)
    y_pred = pd.DataFrame(y_pred)
    y_pred_df = pd.concat([y_pred_df,y_pred],axis=1)
  y_pred = y_pred_df.T.mode().iloc[0]
  
  print(accuracy_score(y_test,y_pred))

df = train_final_model_dt(dfs,unseen_x,unseen_y)

0.5178571428571429


In [248]:
dataframes = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11]
df = train_final_model_dt(dataframes,unseen_x,unseen_y)

NameError: ignored

In [183]:
df.T.mode().iloc[0]

0      2.0
1      1.0
2      1.0
3      1.0
4      2.0
      ... 
109    1.0
110    1.0
111    1.0
112    1.0
113    1.0
Name: 0, Length: 114, dtype: float64

In [146]:
df_arr = df.to_numpy()

array([[2, 2, 2, ..., 2, 1, 2],
       [1, 1, 1, ..., 1, 1, 2],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 2],
       [2, 1, 1, ..., 2, 2, 2]])

In [228]:
# Loading Data
columns = ['t_bilirubin', 'd_bilirubin', 'alkphos','sgpt', 'sgot', 'albumin', 'result']
df = pd.read_csv('/content/drive/MyDrive/0_Hackathon_SBSSU/liver cd/ilpd_md2.csv',usecols=columns)
df = df.astype({'alkphos':'int64','sgpt':'int64','sgot':'int64'})
df.head()

,t_bilirubin,d_bilirubin,alkphos,sgpt,sgot,albumin,result
0,0.7,0.10,187,16,18,3.3,1
1,5.3,2.95,481,64,100,3.2,1
2,5.3,2.95,481,60,68,3.3,1
3,1.0,0.40,182,14,20,3.4,1
4,3.9,2.00,195,27,59,2.4,1
